# Lesson 5 – Extra Practice 

## State Memory & Branching  


A minimal two‑node graph to observe how **state snapshots**, **branching**, and **time‑travel** behave.

### State Fields
* `lnode` – name of the last node that executed  
* `scratch` – arbitrary scratchpad text  
* `count` – integer that **adds** (`operator.add`) on each step


In [1]:
# --- Setup ------------------------------------------------------
from dotenv import load_dotenv
_ = load_dotenv()               # Load secrets from .env

from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langgraph.checkpoint.sqlite import SqliteSaver


## 1. Define the agent *state*

In [2]:
class AgentState(TypedDict):
    lnode: str                                  # node that executed
    scratch: str                                # mutable scratch‑pad
    count: Annotated[int, operator.add]         # adds on merge


## 2. Node callbacks

Each node:

1. Prints its own name and the incoming `count`.
2. Returns a partial state update:
   * `lnode` – its own label (for traceability)
   * `count` – always `1` (which *adds* to the old value)


In [3]:
def node1(state: AgentState):
    # Expect count to start at 0 and increment
    print(f" node1 - entering with count={state['count']}")
    return {"lnode": "node_1", "count": 1}

def node2(state: AgentState):
    print(f" node2 - entering with count={state['count']}")
    return {"lnode": "node_2", "count": 1}


## 3. Wiring the graph – simple loop until `count` ≥ 3

In [4]:
def should_continue(state: AgentState) -> bool:
    """Continue looping while count < 3"""
    return state["count"] < 3

builder = StateGraph(AgentState)
builder.add_node("Node1", node1)
builder.add_node("Node2", node2)

builder.add_edge("Node1", "Node2")                       # fixed edge
builder.add_conditional_edges("Node2", should_continue, {True: "Node1", False: END})
builder.set_entry_point("Node1")

# Checkpointer stores every transition
memory = SqliteSaver.from_conn_string(":memory:")
graph = builder.compile(checkpointer=memory)


## 4. First run (thread 1)

Expected console output:  
`node1 → node2 → node1 → node2`, after which the graph stops.


In [5]:
thread1 = {"configurable": {"thread_id": "1"}}
graph.invoke({"count": 0, "scratch": "hi"}, thread1)


 node1 - entering with count=0
 node2 - entering with count=1
 node1 - entering with count=2
 node2 - entering with count=3


{'lnode': 'node_2', 'scratch': 'hi', 'count': 4}

### Observation

Returned state should contain:

* `lnode: 'node_2'` – last executed node  
* `count: 4` – four increments (0→1→2→3→4)


## 5. Inspecting the latest snapshot

In [6]:
latest = graph.get_state(thread1)
latest


StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 4}, next=(), config={'configurable': {'thread_id': '1', 'thread_ts': '1f02f2fd-94f1-66e6-8004-413af64fdace'}}, metadata={'source': 'loop', 'step': 4, 'writes': {'Node2': {'count': 1, 'lnode': 'node_2'}}}, created_at='2025-05-12T12:51:40.429263+00:00', parent_config={'configurable': {'thread_id': '1', 'thread_ts': '1f02f2fd-94ed-6bc5-8003-385f0873ab38'}})

`latest.next` should be `None` (no further nodes).

In [7]:
print("Next node:", latest.next)

Next node: ()


### 5.1 Full history (newest first)

In [8]:
history1 = list(graph.get_state_history(thread1))

for snap in history1:
    print(snap, "\n")  # Full StateSnapshot
    lnode = snap.values.get('lnode', '—')  # Gracefully handle missing 'lnode'
    print(f"step={snap.metadata['step']:<2}  count={snap.values['count']}  node={lnode}\n")


StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 4}, next=(), config={'configurable': {'thread_id': '1', 'thread_ts': '1f02f2fd-94f1-66e6-8004-413af64fdace'}}, metadata={'source': 'loop', 'step': 4, 'writes': {'Node2': {'count': 1, 'lnode': 'node_2'}}}, created_at='2025-05-12T12:51:40.429263+00:00', parent_config={'configurable': {'thread_id': '1', 'thread_ts': '1f02f2fd-94ed-6bc5-8003-385f0873ab38'}}) 

step=4   count=4  node=node_2

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hi', 'count': 3}, next=('Node2',), config={'configurable': {'thread_id': '1', 'thread_ts': '1f02f2fd-94ed-6bc5-8003-385f0873ab38'}}, metadata={'source': 'loop', 'step': 3, 'writes': {'Node1': {'count': 1, 'lnode': 'node_1'}}}, created_at='2025-05-12T12:51:40.427749+00:00', parent_config={'configurable': {'thread_id': '1', 'thread_ts': '1f02f2fd-94ea-6506-8002-4e56aaaaf0e8'}}) 

step=3   count=3  node=node_1

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 2}, next=('

## 6. Time‑travel: resume from snapshot with `count==1`

Pick the snapshot right after the first `Node1` execution and invoke the graph from there.


In [9]:
travel_cfg = next(s for s in history1 if s.values['count'] == 1)
graph.invoke(None, travel_cfg.config)


 node2 - entering with count=1
 node1 - entering with count=2
 node2 - entering with count=3


{'lnode': 'node_2', 'scratch': 'hi', 'count': 4}

Re‑print history – notice new snapshots (higher `step` numbers) representing the *branch*.

In [10]:
for snap in graph.get_state_history(thread1):
    parent = snap.parent_config.get('configurable', {}).get('thread_ts') if snap.parent_config else "None"
    config = snap.config.get('configurable', {}).get('thread_ts')
    lnode = snap.values.get('lnode', '—')
    print(f"step={snap.metadata['step']:<2}  count={snap.values['count']}  lnode={lnode:<8}  thread_ts={config}  parent_ts={parent}")


step=4   count=4  lnode=node_2    thread_ts=1f02f2fd-9536-6a84-8004-4980f81a6980  parent_ts=1f02f2fd-9533-682c-8003-d45256b9babc
step=3   count=3  lnode=node_1    thread_ts=1f02f2fd-9533-682c-8003-d45256b9babc  parent_ts=1f02f2fd-9530-6667-8002-15ba17184b7b
step=2   count=2  lnode=node_2    thread_ts=1f02f2fd-9530-6667-8002-15ba17184b7b  parent_ts=1f02f2fd-94e3-6d17-8001-a16aa968101b
step=4   count=4  lnode=node_2    thread_ts=1f02f2fd-94f1-66e6-8004-413af64fdace  parent_ts=1f02f2fd-94ed-6bc5-8003-385f0873ab38
step=3   count=3  lnode=node_1    thread_ts=1f02f2fd-94ed-6bc5-8003-385f0873ab38  parent_ts=1f02f2fd-94ea-6506-8002-4e56aaaaf0e8
step=2   count=2  lnode=node_2    thread_ts=1f02f2fd-94ea-6506-8002-4e56aaaaf0e8  parent_ts=1f02f2fd-94e3-6d17-8001-a16aa968101b
step=1   count=1  lnode=node_1    thread_ts=1f02f2fd-94e3-6d17-8001-a16aa968101b  parent_ts=1f02f2fd-94d9-6715-8000-55d988cd1680
step=0   count=0  lnode=—         thread_ts=1f02f2fd-94d9-6715-8000-55d988cd1680  parent_ts=1f02f

## 7. Manual state edits (thread 2)

In [11]:
thread2 = {"configurable": {"thread_id": "2"}}
graph.invoke({"count": 0, "scratch": "hi"}, thread2)

# Choose a mid‑run snapshot
hist2 = list(graph.get_state_history(thread2))
save_state = hist2[-3]                        # snapshot after first Node1

# Patch values
save_state.values["count"] = -3               # will be added
save_state.values["scratch"] = "hello"


 node1 - entering with count=0
 node2 - entering with count=1
 node1 - entering with count=2
 node2 - entering with count=3


### 7.1 Update **without** `as_node` – graph cannot continue

In [12]:
graph.update_state(thread2, save_state.values)
graph.get_state(thread2)


StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hello', 'count': 1}, next=('Node1',), config={'configurable': {'thread_id': '2', 'thread_ts': '1f02f2fd-9568-6f9c-8005-222c00decd26'}}, metadata={'source': 'update', 'step': 5, 'writes': {'Node2': {'count': -3, 'lnode': 'node_1', 'scratch': 'hello'}}}, created_at='2025-05-12T12:51:40.478241+00:00', parent_config={'configurable': {'thread_id': '2', 'thread_ts': '1f02f2fd-955c-6c3e-8004-704dbdf0959b'}})

### 7.2 Update **with** `as_node='Node1'` – graph flows to Node2

In [13]:
graph.update_state(thread2, save_state.values, as_node="Node1")
print("Next node:", graph.get_state(thread2).next)

Next node: ('Node2',)


In [14]:
# Resume execution
graph.invoke(None, thread2)

 node2 - entering with count=-2
 node1 - entering with count=-1
 node2 - entering with count=0
 node1 - entering with count=1
 node2 - entering with count=2


{'lnode': 'node_2', 'scratch': 'hello', 'count': 3}

## 8. Key take‑aways

* Snapshots are immutable – edits create **new** snapshots.
* Use `.config` from any snapshot to *rewind* or *fork* execution.
* Provide `as_node` when patching state so LangGraph can compute the correct next hop.
* Fields annotated with `operator.add` accumulate – be mindful when you hand‑edit them!
